In [1]:
!pip install transformers pandas sentencepiece

In [ ]:
import random
import torch
import pandas as pd
from transformers import T5ForConditionalGeneration, T5Tokenizer


from google.colab import files
uploaded = files.upload()  # Выберите CSV-файл с колонками question, answer, category
file_name = list(uploaded.keys())[0]
faq_df = pd.read_csv(file_name)

Saving vitte_faq.csv to vitte_faq.csv


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "cointegrated/rut5-base-paraphraser"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/315 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/724 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/977M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(30000, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(30000, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
def generate_paraphrases(text, num_return_sequences=1):
    inputs = tokenizer(text, return_tensors="pt", padding=True).to(device)
    max_len = int(inputs.input_ids.shape[1] * 1.5 + 10)
    outputs = model.generate(
        **inputs,
        max_length=max_len,
        num_beams=8,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        early_stopping=True,
        encoder_no_repeat_ngram_size=3
    )
    results = []
    for output in outputs:
        paraphrase = tokenizer.decode(output, skip_special_tokens=True)
        results.append(paraphrase)
    return results


In [5]:
# Шаблонные фразы для вариаций стиля
greetings = ["Привет!", "Здравствуйте!", "Добрый день!", "Эй!", "Салют!"]
emotions = ["Срочно", "Очень нужно", "SOS", "У меня завтра экзамен"]
slang = ["плиз", "пжлста", "ща"]

assistant_greetings = ["Здравствуйте!", "Привет!", "Конечно,", "Пожалуйста,", "Разумеется,"]

dialogue_data = []  # список словарей с полями {'question': ..., 'answer': ...}

for _, row in faq_df.iterrows():
    orig_q = str(row["question"])
    orig_a = str(row["answer"])

    # Для каждой FAQ-строки создаём от 6 до 8 вариантов
    num_variants = random.randint(6, 8)
    for i in range(num_variants):
        # Сгенерировать новые формулировки вопроса и ответа
        new_q = generate_paraphrases(orig_q, num_return_sequences=1)[0]
        new_a = generate_paraphrases(orig_a, num_return_sequences=1)[0]

        # Добавить приветствие к вопросу (с вероятностью 50%)
        if random.random() < 0.5:
            new_q = random.choice(greetings) + " " + new_q

        # Добавить эмоциональную вставку в вопрос (с вероятностью 50%)
        if random.random() < 0.5:
            emo = random.choice(emotions)
            # вставляем эмоцию в начало вопроса
            new_q = emo + ", " + new_q[0].lower() + new_q[1:]

        # Заменить "пожалуйста" на разговорное "плиз" (если встречается)
        new_q = new_q.replace("пожалуйста", random.choice(slang) if random.random() < 0.5 else "пожалуйста")

        # Добавить приветствие/старт к ответу
        if random.random() < 0.5:
            prefix = random.choice(assistant_greetings)
            new_a = prefix + " " + new_a

        # Опционально сделать мини-диалог (две реплики пользователя и две ассистента)
        if random.random() < 0.3:
            # Вторая реплика пользователя (например, благодарность или уточнение)
            follow_up_user = random.choice(["Спасибо, а ещё?", "Очень интересно, а что насчёт ...",
                                           "Понял, а как быть если ...", "А можете добавить что-то?"])
            # Вторая реплика ассистента (например, уточнение или продолжение)
            follow_up_assist = random.choice(["Конечно, вот дополнительная информация: ",
                                             "Разумеется, уточняю: ",
                                             "Да, уточняю: "])
            follow_up_assist += new_a  # повторно используем парафраз ответа или его часть

            # Собираем диалог в одну строку (с разделителем \n)
            question_dialog = new_q + "\n" + follow_up_user
            answer_dialog = new_a + "\n" + follow_up_assist
        else:
            question_dialog = new_q
            answer_dialog = new_a

        dialogue_data.append({"question": question_dialog, "answer": answer_dialog})


In [6]:
# Преобразуем сгенерированные примеры в DataFrame
out_df = pd.DataFrame(dialogue_data)[["question", "answer"]]

# Сохраняем результат в локальный файл
out_df.to_csv("dialog_dataset.csv", index=False, encoding="utf-8-sig")


In [7]:
files.download("dialog_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>